# FB Prophet Model - Unemployment Rate (%)

Training (1997-2020), Training (2021-2023)

# Packages & Data Loading

## Packages

In [1]:
# Import packages
import pandas as pd
from prophet import Prophet

import gdown
import requests
from io import StringIO

from prophet.plot import plot_plotly, plot_components_plotly
import plotly.graph_objects as go

## Data Loading

In [2]:
# Google Drive file
file_id = '117pk3NKZPfYmHQyP452HeMV7_8Jq6_W0'

# Create the download URL
download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Use requests to get file content
response = requests.get(download_url)
response.raise_for_status()  

# Load the CSV directly into df
csv_data = StringIO(response.text)
df = pd.read_csv(csv_data)

# Display data
print('Shape of the dataset:', df.shape)
print(df.dtypes)

Shape of the dataset: (349920, 14)
ref_date                   object
geo                        object
labour_force               object
industry                   object
sex                        object
age_group                  object
value_unemployment        float64
wages                      object
value_wage                float64
year                        int64
month                       int64
ref_date_numeric          float64
value_unemployment_log    float64
value_wage_log            float64
dtype: object


In [3]:
# Create a copy with selected columns
df_var = df[['ref_date', 'value_unemployment']].copy()

print(df_var.head())

     ref_date  value_unemployment
0  1997-01-01                 9.7
1  1997-01-01                 6.3
2  1997-01-01                 7.8
3  1997-01-01                 9.3
4  1997-01-01                 7.5


In [4]:
# Group df_var by 'ref_date' and calculate the median 'value_unemployment'
df_fb = df_var.groupby('ref_date').agg(
    median_unemployment=('value_unemployment', 'median')  # Calculate the median 
).reset_index()

print(df_fb)

       ref_date  median_unemployment
0    1997-01-01            12.916667
1    1997-02-01            10.700000
2    1997-03-01            10.766667
3    1997-04-01            11.925000
4    1997-05-01             9.975000
..          ...                  ...
319  2023-08-01             7.933333
320  2023-09-01             8.048674
321  2023-10-01             7.643939
322  2023-11-01             9.662319
323  2023-12-01             8.497222

[324 rows x 2 columns]


In [5]:
# Display shape and data types 
print('Shape of the dataset:\n', df_var.shape)
print(df_fb.dtypes)

# Convert 'ref_date' to datetime
df_fb['ref_date'] = pd.to_datetime(df_fb['ref_date'])

# Set 'ref_date' as the index 
df_fb = df_fb.set_index('ref_date')

# Verify the index is set and DatetimeIndex
print('\n\nIndex after setting ref_date:\n', df_fb.head().index)
print('\n\nData types after setting index:\n', df_fb.dtypes)
print('\n\nDataset:\n', df_fb.head(2))

Shape of the dataset:
 (349920, 2)
ref_date                object
median_unemployment    float64
dtype: object


Index after setting ref_date:
 DatetimeIndex(['1997-01-01', '1997-02-01', '1997-03-01', '1997-04-01',
               '1997-05-01'],
              dtype='datetime64[ns]', name='ref_date', freq=None)


Data types after setting index:
 median_unemployment    float64
dtype: object


Dataset:
             median_unemployment
ref_date                       
1997-01-01            12.916667
1997-02-01            10.700000


# FB Prophet Model

## Assign Training & Testing Sets

In [6]:
# Split into training and testing sets
train = df_fb['1997-01-01':'2020-12-01']
test = df_fb['2021-01-01':'2023-12-01']

print(train.head())
print(train.tail())

            median_unemployment
ref_date                       
1997-01-01            12.916667
1997-02-01            10.700000
1997-03-01            10.766667
1997-04-01            11.925000
1997-05-01             9.975000
            median_unemployment
ref_date                       
2020-08-01            12.592857
2020-09-01            11.293182
2020-10-01            11.452273
2020-11-01            10.964286
2020-12-01            12.687500


In [7]:
# Prepare data for Prophet
train = train.reset_index()  # Reset index to make 'ref_date' a column
train.rename(columns={'ref_date': 'ds', 'median_unemployment': 'y'}, inplace=True)

# Prepare data for Prophet
test = test.reset_index()  # Reset index to make 'ref_date' a column
test.rename(columns={'ref_date': 'ds', 'median_unemployment': 'y'}, inplace=True)

## Model Fitting

In [8]:
# Fit model
m = Prophet()
m.fit(train)

22:01:32 - cmdstanpy - INFO - Chain [1] start processing
22:01:32 - cmdstanpy - INFO - Chain [1] done processing


## Model Predictions

In [9]:
# Predictions
future = m.make_future_dataframe(periods=36, freq='ME')
future.tail()

,ds
319,2023-07-31
320,2023-08-31
321,2023-09-30
322,2023-10-31
323,2023-11-30


In [10]:
# Prediction columns 
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
319,2023-07-31,11.292113,9.898609,12.800008
320,2023-08-31,10.478593,8.907324,11.940969
321,2023-09-30,9.630695,8.197885,11.123666
322,2023-10-31,11.913959,10.428435,13.511535
323,2023-11-30,10.553859,9.017852,12.046876


# Visualizations

In [11]:
plot_plotly(m, forecast)

In [12]:
# Generate the Prophet forecast plot
fig = plot_plotly(m, forecast)

# Add the test data to the plot
fig.add_trace(
    go.Scatter(
        x=test['ds'],  # x-axis (dates from test data)
        y=test['y'],  # y-axis (actual values)
        mode='markers+lines',
        name='True Values (Test Data)',
        line=dict(color='red', dash='dot'),  # Custom line style
        marker=dict(color='red')
    )
)

# Adjust x-axis range to focus on 2021-2023
fig.update_layout(
    xaxis=dict(
        range=['2021-01-01', '2023-12-31'],  # Set x-axis range
        title='Date',  # Add x-axis title
    ),
    yaxis=dict(
        title='Median Unemployment Rate',  # Add y-axis title
    ),
    title='Prophet Forecast with Test Data (2021-2023)',  # Add plot title
)

# Show the combined plot
fig.show()


In [13]:
plot_components_plotly(m, forecast)